In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('/content/base_low_default.csv')
df.head()

,renda_mil,idade,score_credito,default
0,9.49,64,807.0,0
1,7.59,69,785.0,0
2,9.94,60,708.0,0
3,12.57,31,831.0,0
4,7.30,23,610.0,0


In [22]:
#Define as variáveis
X = df[['renda_mil', 'idade', 'score_credito']]
y = df['default']

In [23]:
#Modelo de Isolation Forest

import numpy as np
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=0.005, random_state=42)
iso.fit(X)

df['anomaly_iso'] = np.where(iso.predict(X) == 1, 0, 1)

y_pred_iso = df['anomaly_iso']



In [24]:
#Modelo de XGBoost com Peso Balanceado

from xgboost import XGBClassifier

peso_balanceado = (y == 0).sum() / (y == 1).sum()

xgb = XGBClassifier(scale_pos_weight=peso_balanceado, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X, y)

y_pred_xgb = xgb.predict(X)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [14:15:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [25]:
#SMOTE + Random Forest
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

sm = SMOTE(random_state=42, k_neighbors=2)
X_res, y_res = sm.fit_resample(X, y)

rf = RandomForestClassifier(random_state=42)
rf.fit(X_res, y_res)

y_pred_rf = rf.predict(X)


In [26]:
#Gerar as métricas
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score
)

def avaliar(y_true, y_pred, metodo):
    cm = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f'\n🔹 {metodo}')
    print('Confusion Matrix:')
    print(cm)
    print(f'Precision: {precision:.4f}')
    print(f'Recall:    {recall:.4f}')
    print(f'F1 Score:  {f1:.4f}')

#Comparar os resultados
avaliar(y, y_pred_iso, 'Isolation Forest')
avaliar(y, y_pred_xgb, 'XGBoost Weighted')
avaliar(y, y_pred_rf, 'SMOTE + RandomForest')




🔹 Isolation Forest
Confusion Matrix:
[[990   5]
 [  5   0]]
Precision: 0.0000
Recall:    0.0000
F1 Score:  0.0000

🔹 XGBoost Weighted
Confusion Matrix:
[[995   0]
 [  0   5]]
Precision: 1.0000
Recall:    1.0000
F1 Score:  1.0000

🔹 SMOTE + RandomForest
Confusion Matrix:
[[995   0]
 [  0   5]]
Precision: 1.0000
Recall:    1.0000
F1 Score:  1.0000


# 🇬🇧 Supervised vs. Unsupervised Learning and the Default Issue

**Supervised Learning:**

Supervised learning models are trained on a labeled dataset, meaning that the data includes both the input features (X) and the corresponding output labels (y). The goal of supervised learning is to learn a mapping from the input features to the output labels so that the model can predict the output for new, unseen data. In this notebook, XGBoost and Random Forest are examples of supervised learning models because they are trained on the `X` (features) and `y` (default label).

**Unsupervised Learning:**

Unsupervised learning models are trained on an unlabeled dataset, meaning that the data only includes the input features (X) and no corresponding output labels. The goal of unsupervised learning is to find patterns and structure within the data. Isolation Forest in this notebook is an example of an unsupervised learning model. It identifies anomalies (potential defaults) without being explicitly trained on default labels.

**The Default Issue (Imbalanced Data):**

The "default issue" often refers to dealing with imbalanced datasets, where the number of instances in one class (e.g., customers who default) is significantly smaller than the number of instances in the other class (e.g., customers who do not default). This imbalance can cause supervised models to be biased towards the majority class, leading to poor performance in identifying the minority class (defaults).

Methods like using `scale_pos_weight` in XGBoost or employing techniques like SMOTE with Random Forest are strategies to address this imbalance in supervised learning by giving more importance to the minority class or creating synthetic samples of the minority class. Unsupervised methods like Isolation Forest can be less affected by class imbalance as they focus on identifying outliers based on the data distribution.

**Why these methods for small datasets?**

For small datasets, complex models with many parameters can easily overfit the training data, meaning they perform very well on the training data but poorly on unseen data. The models used here, while powerful, can be more manageable with smaller datasets compared to very deep neural networks, for example. Additionally, techniques like SMOTE can help to effectively increase the size and diversity of the minority class in smaller datasets, which is crucial for training supervised models effectively when dealing with imbalance.

========================================================================





# 🇧🇷 Aprendizado Supervisionado vs. Não Supervisionado e o Problema do Default



**Aprendizado Supervisionado:**

Modelos de aprendizado supervisionado são treinados em um conjunto de dados rotulado, o que significa que os dados incluem tanto as características de entrada (X) quanto os rótulos de saída correspondentes (y). O objetivo do aprendizado supervisionado é aprender um mapeamento das características de entrada para os rótulos de saída para que o modelo possa prever a saída para novos dados não vistos. Neste notebook, XGBoost e Random Forest são exemplos de modelos de aprendizado supervisionado porque são treinados nos dados `X` (características) e `y` (rótulo de default).

**Aprendizado Não Supervisionado:**

Modelos de aprendizado não supervisionado são treinados em um conjunto de dados não rotulado, o que significa que os dados incluem apenas as características de entrada (X) e nenhum rótulo de saída correspondente. O objetivo do aprendizado não supervisionado é encontrar padrões e estrutura dentro dos dados. O Isolation Forest neste notebook é um exemplo de modelo de aprendizado não supervisionado. Ele identifica anomalias (potenciais defaults) sem ser explicitamente treinado em rótulos de default.

**O Problema do Default (Dados Desbalanceados):**

O "problema do default" geralmente se refere a lidar com conjuntos de dados desbalanceados, onde o número de instâncias em uma classe (por exemplo, clientes que dão default) é significativamente menor do que o número de instâncias na outra classe (por exemplo, clientes que não dão default). Esse desbalanceamento pode fazer com que os modelos supervisionados sejam tendenciosos para a classe majoritária, levando a um desempenho ruim na identificação da classe minoritária (defaults).

Métodos como usar `scale_pos_weight` no XGBoost ou empregar técnicas como SMOTE com Random Forest são estratégias para lidar com esse desbalanceamento no aprendizado supervisionado, dando mais importância à classe minoritária ou criando amostras sintéticas da classe minoritária. Métodos não supervisionados como o Isolation Forest podem ser menos afetados pelo desbalanceamento de classes, pois se concentram em identificar outliers com base na distribuição dos dados.

**Por que esses métodos para pequenos conjuntos de dados?**

Para conjuntos de dados pequenos, modelos complexos com muitos parâmetros podem facilmente sobreajustar os dados de treinamento, o que significa que eles têm um desempenho muito bom nos dados de treinamento, mas um desempenho ruim em dados não vistos. Os modelos usados aqui, embora poderosos, podem ser mais gerenciáveis com conjuntos de dados menores em comparação com redes neurais muito profundas, por exemplo. Além disso, técnicas como SMOTE podem ajudar a aumentar efetivamente o tamanho e a diversidade da classe minoritária em conjuntos de dados menores, o que é crucial para treinar modelos supervisionados de forma eficaz ao lidar com desbalanceamento.

